<a href="https://colab.research.google.com/github/MDfarazuddin99/Face_Masked_Recognition/blob/master/Mask_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-l3eb5233
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-l3eb5233
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp36-none-any.whl size=8311 sha256=a4bc7e78747571dc1ef9ec54c79f26669939b7c86d2bb1b0314840d53a5be20b
  Stored in directory: /tmp/pip-ephem-wheel-cache-ot6u2xxc/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [0]:
pip install keras_vggface

In [0]:
from keras_vggface.vggface import VGGFace

# Based on VGG16 architecture -> old paper(2015)
vggface = VGGFace(model='vgg16') # or VGGFace() as default

Using TensorFlow backend.


In [0]:
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input
from keras_vggface.vggface import VGGFace

#custom parameters
nb_class = 4

hidden_dim = 256

vgg_model = VGGFace(include_top=False, input_shape=(128, 128, 3),pooling='avg')
last_layer = vgg_model.get_layer('pool5').output
x = Flatten(name='flatten')(last_layer)
x = Dense(hidden_dim, activation='relu', name='fc6')(x)
x = Dense(hidden_dim, activation='relu', name='fc7')(x)
out = Dense(nb_class, activation='softmax', name='fc8')(x)
custom_vgg_model = Model(vgg_model.input, out)


In [0]:
layer_count = 0
for layer in custom_vgg_model.layers:
	layer_count = layer_count+1

for l in range(layer_count-3):
	custom_vgg_model.layers[l].trainable=False
    

In [0]:
custom_vgg_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'],)

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras_vggface import utils
preprocess_input = utils.preprocess_input
datagenerator = ImageDataGenerator(preprocessing_function = preprocess_input)

In [0]:
image_size = 128
train_generator = datagenerator.flow_from_directory(
        '/content/drive/My Drive/5-celebrity-faces-dataset/custom_data/train',
        target_size=(image_size, image_size),
        batch_size=32,
        class_mode='categorical',)

validation_generator = datagenerator.flow_from_directory(
        '/content/drive/My Drive/5-celebrity-faces-dataset/custom_data/test',
        target_size=(image_size, image_size),
        class_mode='categorical')

custom_vgg_model.fit_generator(
        train_generator,
        validation_data=validation_generator,
        validation_steps=2,epochs=20)

Found 71 images belonging to 4 classes.
Found 20 images belonging to 4 classes.
Epoch 1/20
3/3 [==============================] - 1s 188ms/step - loss: 4.3897e-04 - accuracy: 1.0000 - val_loss: 13.4970 - val_accuracy: 0.6500
Epoch 2/20
3/3 [==============================] - 0s 166ms/step - loss: 2.7646e-05 - accuracy: 1.0000 - val_loss: 14.0070 - val_accuracy: 0.6500
Epoch 3/20
3/3 [==============================] - 0s 163ms/step - loss: 4.0773e-06 - accuracy: 1.0000 - val_loss: 14.3881 - val_accuracy: 0.6500
Epoch 4/20
3/3 [==============================] - 0s 144ms/step - loss: 1.8185e-06 - accuracy: 1.0000 - val_loss: 14.7261 - val_accuracy: 0.6500
Epoch 5/20
3/3 [==============================] - 0s 155ms/step - loss: 1.6658e-06 - accuracy: 1.0000 - val_loss: 14.9924 - val_accuracy: 0.6500
Epoch 6/20
1/3 [=========>....................] - ETA: 0s - loss: 3.2782e-07 - accuracy: 1.0000

In [0]:
from keras.preprocessing import image
import numpy as np
test_path = input()
img = image.load_img(test_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = utils.preprocess_input(x, version=1) # or version=2
preds = custom_vgg_model.predict(x)
print(preds)
if preds[0][0]>preds[0][1]:
    print('Faraz')
else:
    print('Farha')